In [4]:
# credits: https://towardsdatascience.com/pyspark-and-xgboost-integration-tested-on-the-kaggle-titanic-dataset-4e75a568bdb

In [6]:
cd ..

/home/user/Documents/workspace/projects/accident-prediction-montreal


In [7]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.types import *
from pyspark.sql.functions import col
import numpy as np
from pyspark.sql.functions import col
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.sql.types import ArrayType
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import os
import shutil
from evaluate import *
from random_forest import *
from utils import init_spark
from preprocess import get_positive_samples, \
                       get_negative_samples, \
                       get_dataset_df
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars data/xgboost4j-spark-0.72.jar,data/xgboost4j-0.72.jar pyspark-shell'

In [8]:
spark = init_spark()
spark.sparkContext.addPyFile("data/sparkxgb.zip")
from sparkxgb import XGBoostEstimator

Spark Session created
Parameters:
	spark.jars: file:///home/user/Documents/workspace/projects/accident-prediction-montreal/data/xgboost4j-spark-0.72.jar,file:///home/user/Documents/workspace/projects/accident-prediction-montreal/data/xgboost4j-0.72.jar
	spark.executor.id: driver
	spark.driver.port: 42495
	spark.driver.host: 192.168.100.49
	spark.app.name: Accident prediction
	spark.serializer: org.apache.spark.serializer.KryoSerializer
	spark.repl.local.jars: file:///home/user/Documents/workspace/projects/accident-prediction-montreal/data/xgboost4j-spark-0.72.jar,file:///home/user/Documents/workspace/projects/accident-prediction-montreal/data/xgboost4j-0.72.jar
	spark.driver.memory: 3g
	spark.rdd.compress: True
	spark.app.id: local-1555304588938
	spark.serializer.objectStreamReset: 100
	spark.master: local[*]
	spark.submit.deployMode: client
	spark.ui.showConsoleProgress: true


In [9]:
# load dataset
sample_ratio = 0.1
neg_samples = get_negative_samples(spark).sample(sample_ratio).na.fill(0)
pos_samples = get_positive_samples(spark).sample(sample_ratio).na.fill(0)
df = get_dataset_df(spark, pos_samples, neg_samples).na.fill(0)
trainDF, testDF = df.randomSplit([0.8, 0.2], seed=0)

In [10]:
xgboost = XGBoostEstimator(
    featuresCol="features", 
    labelCol="label", 
    predictionCol="prediction"
)
pipeline = Pipeline().setStages([xgboost])

In [11]:
model = pipeline.fit(trainDF)

In [ ]:
def save_model():
    workdir = "./"
    path = "data/xgboost.model"
    if os.path.isdir(path):
        shutil.rmtree(path)
    model.save(path)

In [12]:
prediction = model.transform(testDF) 
prediction = prediction.withColumn("rawPrediction", prediction['probabilities'])

In [ ]:
# plot graph
"""area_under_PR, f1_score = evaluate_binary_classifier(prediction)"""
"""pd_df = compute_precision_recall_graph(result_df, 20)
pd_df.plot()"""

In [ ]:
paramGrid = (ParamGridBuilder().addGrid(xgboost.max_depth, [x for x in range(3, 20, 6)])
                            .addGrid(xgboost.eta, [x for x in np.linspace(0.2, 0.6, 4)])
                            .addGrid(xgboost.scale_pos_weight, [x for x in np.linspace(0.03, 1.0, 12)])
                             .build())

evaluator = BinaryClassificationEvaluator(labelCol="label",
                                      rawPredictionCol="probabilities",
                                      metricName="areaUnderPR")

cv = (CrossValidator()
        .setEstimator(pipeline)
        .setEvaluator(evaluator)
        .setEstimatorParamMaps(paramGrid)
        .setNumFolds(3))

cvModel = cv.fit(trainDF)

bestModel = (cvModel.bestModel
                    .asInstanceOf[PipelineModel]
                    .stages(2)
                    .asInstanceOf[XGBoostClassificationModel])

bestModel.extractParamMap()

In [ ]:
# save_model()